In [1]:
#data preprocessing
import numpy as np
import pyodbc
import pandas as pd

#ml stuff
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow import keras 
import tensorflow as tf
print('Tensorflow/Keras: %s' % keras.__version__)
from keras.models import Sequential 
from keras import Input 
from keras.layers import Dense
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

#utils/visualization
import time
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import dill as pickle
from ydata_profiling import ProfileReport
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import itertools
import functools
import operator 
from more_itertools import flatten
from collections import Counter
#config vars
pd.options.display.max_columns=1000
pd.options.display.max_rows = 10000
pd.options.display.max_seq_items = 100
verbose=1

from sqlalchemy.engine import URL
from sqlalchemy import create_engine

import sqlalchemy as sa
import urllib
from sqlalchemy import text

Tensorflow/Keras: 2.12.0


# Connect to Database and get Data

In [17]:
# OLD and slower
def connect_to_db(connection_string,
                    database,
                    driver = 'SQL Server Native Client 11.0',
                    user = 'CCAdmin',
                    password = 'Miw6RjnTGmPHLYF9mG1o'
):
    connection = pyodbc.connect("Driver={"+driver+"};"
                        "Server="+connection_string+";"
                        "Database="+database+";"
                        "uid="+user+";pwd="+password+"")
    return connection

def get_table_data(table_name, connection):
    query = "SELECT * FROM {}".format(table_name)
    df = pd.read_sql_query(query, connection)

    return df


df_table_cclogattributes = get_table_data('CCLogAttributes', connect_to_db(connection_string= 'classconprocessingger.database.windows.net', database = 'T_'+tenant+''))
df_table_cclogattributes = df_table_cclogattributes.drop(['Zone','Attribute_DataType','LogTimeTicks'], axis=1)

df_table_cclogattributes = df_table_cclogattributes.replace('\n',' ', regex=True)
df_table_cclogattributes = df_table_cclogattributes.replace('\r',' ', regex=True)
df_table_cclogattributes.to_csv('data/cclogattributes_T_'+tenant+'.csv', index=False, header= True, encoding='utf-8')#iso-8859-15


C:\Users\lete\AppData\Local\Temp\ipykernel_10816\2088681304.py:16: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [126]:
tenant = '0006sg'

In [54]:
def connect_to_db_better(connection_string,
                    database,
                    driver = 'SQL Server Native Client 11.0',
                    user = 'CCAdmin',
                    password = 'Miw6RjnTGmPHLYF9mG1o'
):
    odbc_str = 'DRIVER='+driver+';SERVER='+connection_string+';PORT=1433;UID='+user+';DATABASE='+ database + ';PWD='+ password
    connect_str = 'mssql+pyodbc:///?odbc_connect=' + urllib.parse.quote_plus(odbc_str)

    return connect_str


def get_table_data_CCLOG(table_name, connect_str, startdate, enddate):
    engine = create_engine(connect_str)
    with engine.connect() as conn:
        df = pd.read_sql(text("SELECT * FROM [dbo].["+table_name+"] where LogTime >= "+startdate+" and LogTime <= "+enddate+""), conn)
        return df
    
def get_table_data_CCLOGAUTOTRAIN(table_name, connect_str, startdate, enddate):
    engine = create_engine(connect_str)
    with engine.connect() as conn:
        df = pd.read_sql(text("SELECT * FROM [dbo].["+table_name+"] where CONVERT(datetime2, STOP_TIME, 104) BETWEEN "+startdate+" and "+enddate+""), conn)
        return df

def get_table_data_ALL(table_name, connect_str):
    engine = create_engine(connect_str)
    with engine.connect() as conn:
        df = pd.read_sql(text("SELECT * FROM [dbo].["+table_name+"]"), conn)
        return df

# Data analysis


In [44]:
#df.head()
#ProfileReport(df)

In [ ]:
dist = df['Delta'].value_counts()
trace = go.Pie(values=(np.array(dist)),labels=dist.index,  pull=[0, 0])
layout = go.Layout(title='Delta Distribution')
data = [trace]
fig = go.Figure(trace,layout)
fig.update_traces(marker=dict(line=dict(color='#000000', width=0.5)), textinfo='value+percent', insidetextorientation='auto')
#sfig.show()

In [ ]:
dist = df['Type'].value_counts()
trace = go.Pie(values=(np.array(dist)),labels=dist.index,  pull=[0, 0.4,0.2,0.1])
layout = go.Layout(title='Type Distribution')
data = [trace]
fig = go.Figure(trace,layout)
fig.update_traces(marker=dict(line=dict(color='#000000', width=0.5)), textinfo='value+percent', insidetextorientation='auto')
#sfig.show()

# Sollerkennungswerte bestimmen


In [5]:
def get_sollwerte(key, df):
    df_ordernum = df.loc[(df['Attribute_Name'] == 'OrderNum')] #&(df['Delta'] == False)
    dist = df_ordernum['Delta'].value_counts(normalize=True)
    print('Kreditor: '+key+'')
    
    try:
        score = 1 - dist.loc[True]
    except:
        score = dist.loc[False]

    print('q(OrderNum): '+str(score)+'') 


    df_ordernum_correct = df.loc[(df['Attribute_Name'] == 'OrderNum') &(df['Delta'] == False)] 
    documentid_list = df_ordernum_correct[['DocumentID']].values.tolist()
    merged = list(itertools.chain.from_iterable(documentid_list))
    denonimator_ordernum = len(documentid_list)
    

    df_debitor_ordernum = df.loc[(df['Attribute_Name'] == 'OrderNum')|(df['Attribute_Name'] == 'DEBITOR_NUM')|(df['Attribute_Name'] == 'VENDOR_NUM')]
    #print(df_debitor_ordernum)
    #df_debitor_ordernum.to_csv('data/debitor_preprocessing.csv', index=False, header= True, encoding='iso-8859-15')
    


    counter_do=0
    counter_vo=0
    for entry_1 in merged:
        speicher_entry_new = df_debitor_ordernum[df_debitor_ordernum.DocumentID == ''.join(str(entry_1))]
        if ((speicher_entry_new.Attribute_Name == 'OrderNum') & (speicher_entry_new.Delta == False)).any(): 
            if ((speicher_entry_new.Attribute_Name == 'DEBITOR_NUM')&(speicher_entry_new.Delta == False)).any():
                    counter_do += 1
            if ((speicher_entry_new.Attribute_Name == 'VENDOR_NUM')&(speicher_entry_new.Delta == False)).any():
                    counter_vo += 1
    try:
        print('q(Mandant|OrderNum): '+str(counter_do/denonimator_ordernum)+'')
        print('q(Vendor|OrderNum): '+str(counter_vo/denonimator_ordernum)+'')
    except:
         print('error')

    counter_vno=0

    df_order_delta = df.loc[(df['Attribute_Name'] == 'OrderNum')&(df['Delta'] == True)]
    df_order_delta_documentid = df_order_delta[['DocumentID']]
    order_delta_list = df_order_delta_documentid.values.tolist()
    order_delta_list_merged = list(itertools.chain.from_iterable(order_delta_list))
    denominator_order_delta_num = len(order_delta_list_merged)


    df_invoicenumber_order_delta = df.loc[(df['Attribute_Name'] == 'DEBITOR_NUM')|(df['Attribute_Name'] == 'OrderNum')]
    for entry_order_delta_invoicenumber in order_delta_list_merged:
        speicher_entry_order_delta = df_invoicenumber_order_delta[df_invoicenumber_order_delta.DocumentID == ''.join(str(entry_order_delta_invoicenumber))]
        if ((speicher_entry_order_delta.Attribute_Name == 'OrderNum') & (speicher_entry_order_delta.Delta == True)).any(): 
            if ((speicher_entry_order_delta.Attribute_Name == 'DEBITOR_NUM') & (speicher_entry_order_delta.Delta == False)).any():
                    counter_vno += 1

    if(counter_vno == 0):
        print('Da keine falsche Ordernum existiert ist Mandant|!Ordernum 0')
    else:
        print('q(Mandant|!OrderNum): '+str(counter_vno/denominator_order_delta_num)+'')
        
    df_vendor_nodelta = df.loc[(df['Attribute_Name'] == 'VENDOR_NUM')&(df['Delta'] == False)]
    df_vendor_documentid = df_vendor_nodelta[['DocumentID']]
    vendor_list = df_vendor_documentid.values.tolist()
    vendor_list_merged = list(itertools.chain.from_iterable(vendor_list))
    denominator_vendor_num = len(vendor_list)

    df_invoicenumber_vendor = df.loc[(df['Attribute_Name'] == 'InvoiceNumber')|(df['Attribute_Name'] == 'VENDOR_NUM')|(df['Attribute_Name'] == 'InvoiceDate')|(df['Attribute_Name'] == 'InvoiceDate')|(df['Attribute_Name'] == 'GrossAmount')]

    counter_inv=0
    counter_idv=0
    counter_gv=0

    for entry_vendor_invoicenumber in vendor_list_merged:
        speicher_entry_vendor = df_invoicenumber_vendor[df_invoicenumber_vendor.DocumentID == ''.join(str(entry_vendor_invoicenumber))]
        if ((speicher_entry_vendor.Attribute_Name == 'VENDOR_NUM') & (speicher_entry_vendor.Delta == False)).any(): 
            if ((speicher_entry_vendor.Attribute_Name == 'InvoiceNumber') & (speicher_entry_vendor.Delta == False)).any():
                    counter_inv += 1
            if ((speicher_entry_vendor.Attribute_Name == 'InvoiceDate') & (speicher_entry_vendor.Delta == False)).any():
                    counter_idv += 1
            if ((speicher_entry_vendor.Attribute_Name == 'GrossAmount') & (speicher_entry_vendor.Delta == False)).any():
                    counter_gv += 1

    try:
        print('q(InvoiceNumber|Vendor): '+str(counter_inv/denominator_vendor_num)+'')
        print('q(InvoiceDate|Vendor): '+str(counter_idv/denominator_vendor_num)+'')
        print('q(Gross|Vendor): '+str(counter_gv/denominator_vendor_num)+'')
    except:
        print('error')
    counter_gnv=0

    df_vendor_delta = df.loc[(df['Attribute_Name'] == 'VENDOR_NUM')&(df['Delta'] == True)]
    df_vendor_delta_documentid = df_vendor_delta[['DocumentID']]
    vendor_delta_list = df_vendor_delta_documentid.values.tolist()
    vendor_delta_list_merged = list(itertools.chain.from_iterable(vendor_delta_list))
    denominator_vendor_delta_num = len(vendor_delta_list_merged)


    df_invoicenumber_vendor_delta = df.loc[(df['Attribute_Name'] == 'GrossAmount')|(df['Attribute_Name'] == 'VENDOR_NUM')]
    for entry_vendor_delta_invoicenumber in vendor_delta_list_merged:
        speicher_entry_vendor_delta = df_invoicenumber_vendor_delta[df_invoicenumber_vendor_delta.DocumentID == ''.join(str(entry_vendor_delta_invoicenumber))]
        if ((speicher_entry_vendor_delta.Attribute_Name == 'VENDOR_NUM') & (speicher_entry_vendor_delta.Delta == True)).any(): 
            if ((speicher_entry_vendor_delta.Attribute_Name == 'GrossAmount') & (speicher_entry_vendor_delta.Delta == False)).any():
                    counter_gnv += 1

    try:
        print('q(Gross|!Vendor): '+str(counter_gnv/denominator_vendor_delta_num)+'')
    except:
        print('errror')

    counter_vom=0

    df_order_mandant_delta = df.loc[(df['Attribute_Name'] == 'OrderNum')&(df['Delta'] == True)]
    df_order_delta_new = df_order_mandant_delta.loc[(df_order_mandant_delta['Attribute_Name'] == 'Mandant')&(df['Delta'] == False)]
    df_order_delta_mandant_documentid = df_order_delta_new[['DocumentID']]
    order_delta_mandant_list = df_order_delta_mandant_documentid.values.tolist()
    order_delta_mandant_list_merged = list(itertools.chain.from_iterable(order_delta_mandant_list))
    denominator_order_delta_mandant_num = len(order_delta_mandant_list_merged)


    df_invoicenumber_order_delta_mandant = df.loc[(df['Attribute_Name'] == 'VENDOR_NUM')|(df['Attribute_Name'] == 'OrderNum')|(df['Attribute_Name'] == 'DEBITOR_NUM')]
    for entry_order_delta_mandant in order_delta_mandant_list_merged:
        speicher_entry_order_delta_mandant = df_invoicenumber_order_delta_mandant[df_invoicenumber_order_delta_mandant.DocumentID == ''.join(str(entry_order_delta_mandant))]
        if (((speicher_entry_order_delta_mandant.Attribute_Name == 'OrderNum') & (speicher_entry_order_delta_mandant.Delta == True))&((speicher_entry_order_delta_mandant.Attribute_Name == 'DEBITOR_NUM') & (speicher_entry_order_delta_mandant.Delta == False))).any(): 
            if ((speicher_entry_order_delta_mandant.Attribute_Name == 'VENDOR_NUM') & (speicher_entry_order_delta_mandant.Delta == False)).any():
                    counter_vom += 1

    if((counter_vom == 0) | (denominator_order_delta_mandant_num == 0 & counter_vom == 0)):
        print('Nulldivision q(Vendor| !OrderNum & Mandant) ungültig')
    else:
        print('q(Vendor| !OrderNum & Mandant): '+str(counter_vom/denominator_order_delta_mandant_num)+'')


    counter_vnom=0

    df_order_mandant_delta = df.loc[(df['Attribute_Name'] == 'OrderNum')&(df['Delta'] == True)]
    df_order_delta_new = df_order_mandant_delta.loc[(df_order_mandant_delta['Attribute_Name'] == 'Mandant')&(df['Delta'] == True)]
    df_order_delta_mandant_documentid = df_order_delta_new[['DocumentID']]
    order_delta_mandant_list = df_order_delta_mandant_documentid.values.tolist()
    order_delta_mandant_list_merged = list(itertools.chain.from_iterable(order_delta_mandant_list))
    denominator_order_ndelta_mandant_num = len(order_delta_mandant_list_merged)


    df_invoicenumber_order_delta_mandant = df.loc[(df['Attribute_Name'] == 'VENDOR_NUM')|(df['Attribute_Name'] == 'OrderNum')|(df['Attribute_Name'] == 'DEBITOR_NUM')]
    for entry_order_delta_mandant in order_delta_mandant_list_merged:
        speicher_entry_order_delta_mandant = df_invoicenumber_order_delta_mandant[df_invoicenumber_order_delta_mandant.DocumentID == ''.join(str(entry_order_delta_mandant))]
        if (((speicher_entry_order_delta_mandant.Attribute_Name == 'OrderNum') & (speicher_entry_order_delta_mandant.Delta == True))&((speicher_entry_order_delta_mandant.Attribute_Name == 'DEBITOR_NUM') & (speicher_entry_order_delta_mandant.Delta == True))).any(): 
            if ((speicher_entry_order_delta_mandant.Attribute_Name == 'VENDOR_NUM') & (speicher_entry_order_delta_mandant.Delta == False)).any():
                    counter_vnom += 1

    if((counter_vnom == 0) | (denominator_order_ndelta_mandant_num == 0)):
        print('Nulldivision q(Vendor| !OrderNum & !Mandant) ungültig')
    else:
        print('q(Vendor| !OrderNum & !Mandant): '+str(counter_vnom/denominator_order_ndelta_mandant_num)+'')
    
    print('--------------------------------------------------------------------------------------------')


# Data analysis


In [4]:
# Nach Kreditoren sortieren
#Nach Zeit sortieren 
def split_datframe_into_whatever(splitkey, df_new):
    df_split = df_new.loc[(df_new['Attribute_Name'] == splitkey)] 

    def split_dataframe(df_debitornum, column):
        split_dfs = {}
        for value in df_debitornum[column].unique():
            split_dfs[value] = df_debitornum[df_debitornum[column] == value][['DocumentID']]   
        return split_dfs

    dc_split_attribute_after = split_dataframe(df_split, 'Attribute_After')


    def find_corresponding_values(dict_of_dataframes, column_to_match, large_dataframe):
        result_dict = {}
        for key, df in dict_of_dataframes.items():
            temp_df = large_dataframe[large_dataframe[column_to_match].isin(df[column_to_match])]
            result_dict[key] = temp_df
        return result_dict

    dc_sorted_df = find_corresponding_values(dc_split_attribute_after, "DocumentID", df_new)
    return dc_sorted_df
    #dc_sorted_df_by_creditor[""].to_csv('data/codia.csv', index=False, header= True, encoding='utf-8')


In [5]:
def filter_df_by_time(df, start_date, end_date):
    # Filtern des DataFrames nach dem Zeitfenster
    filtered_df = df[(df['LogTime'] >= start_date) & (df['LogTime'] <= end_date)]  
    df_new = filtered_df.drop(['LogTime'], axis =1)
    # Rückgabe des gefilterten DataFrames
    df_new.to_csv('data/cclogattributes_T_'+tenant+'_reduced.csv', index=False, header= True, encoding='utf-8')#iso-8859-15
    return df_new

In [6]:
def get_data_values(df, value1, bool1, value2, bool2): 
    df_ordernum_correct = df.loc[(df['Attribute_Name'] == value1) &(df['Delta'] == bool1)] 
    documentid_list = df_ordernum_correct[['DocumentID']].values.tolist()
    merged = list(itertools.chain.from_iterable(documentid_list))
    denonimator_ordernum = len(documentid_list)
    
    df_debitor_ordernum = df.loc[(df['Attribute_Name'] == value1)|(df['Attribute_Name'] == value2)]

    counter_do=0
    for entry_1 in merged:
        speicher_entry_new = df_debitor_ordernum[df_debitor_ordernum.DocumentID == ''.join(str(entry_1))]
        if ((speicher_entry_new.Attribute_Name == value1) & (speicher_entry_new.Delta == bool1)).any(): 
            if ((speicher_entry_new.Attribute_Name == value2)&(speicher_entry_new.Delta == bool2)).any():
                    counter_do += 1
                    x = speicher_entry_new["DocumentID"]

    try:
        return counter_do/denonimator_ordernum , x
        #return ([counter_do/denonimator_ordernum,counter_do,denonimator_ordernum])
    except:
         return pd.NA, pd.NA

In [7]:
def get_data_values2(df, items): 
    df_ordernum_correct = df.loc[(df['Attribute_Name'] == items[0]) &(df['Delta'] == items[1])] 
    documentid_list = df_ordernum_correct[['DocumentID']].values.tolist()
    merged = list(itertools.chain.from_iterable(documentid_list))
    denominator_ordernum = len(documentid_list)
    
    df_debitor_ordernum = df.loc[(df['Attribute_Name'] == items[0])|(df['Attribute_Name'] == items[2])]

    counter_do=0
    for entry_1 in merged:
        speicher_entry_new = df_debitor_ordernum[df_debitor_ordernum.DocumentID == ''.join(str(entry_1))]
        if ((speicher_entry_new.Attribute_Name == items[0]) & (speicher_entry_new.Delta == items[1])).any(): 
            if ((speicher_entry_new.Attribute_Name == items[2])&(speicher_entry_new.Delta == False)).any():
                    counter_do += 1


    try:
        return counter_do/denominator_ordernum,denominator_ordernum
    except ZeroDivisionError:
         return pd.NA, pd.NA

In [8]:
def get_data_values4(df, items):
    df_filtered = df[(df['Attribute_Name'] == items[0]) & (df['Delta'] == items[1])]
    documentid_list = df_filtered['DocumentID'].tolist()
    denominator_ordernum = len(documentid_list)
    
    df_debitor_ordernum = df[df['Attribute_Name'].isin([items[0], items[2]])]
    
    mask = (df_debitor_ordernum['DocumentID'].isin(documentid_list)) & \
           (df_debitor_ordernum['Attribute_Name'].isin([items[0], items[2]])) & \
           (df_debitor_ordernum['Delta'].isin([items[1], False]))
    
    counter_do = df_debitor_ordernum[mask].groupby('DocumentID').apply(lambda x: ((x['Attribute_Name'] == items[0]) & (x['Delta'] == items[1])).any() and ((x['Attribute_Name'] == items[2]) & (x['Delta'] == False)).any()).sum()

    try:
        return int(counter_do) / denominator_ordernum, denominator_ordernum
    except ZeroDivisionError:
        return pd.NA, pd.NA

In [8]:
def get_data_values_two_booleans(df, value1, bool1, value2, bool2, value3):    
    counter_vom=0

    df_order_mandant_delta = df.loc[(df['Attribute_Name'] == value1)&(df['Delta'] == bool1)]
    df_order_delta_new = df_order_mandant_delta.loc[(df_order_mandant_delta['Attribute_Name'] == value2)&(df['Delta'] == bool2)]
    df_order_delta_mandant_documentid = df_order_delta_new[['DocumentID']]
    order_delta_mandant_list = df_order_delta_mandant_documentid.values.tolist()
    order_delta_mandant_list_merged = list(itertools.chain.from_iterable(order_delta_mandant_list))
    denominator_order_delta_mandant_num = len(order_delta_mandant_list_merged)


    df_invoicenumber_order_delta_mandant = df.loc[(df['Attribute_Name'] == value1)|(df['Attribute_Name'] == value2)|(df['Attribute_Name'] == value3)]
    for entry_order_delta_mandant in order_delta_mandant_list_merged:
        speicher_entry_order_delta_mandant = df_invoicenumber_order_delta_mandant[df_invoicenumber_order_delta_mandant.DocumentID == ''.join(str(entry_order_delta_mandant))]
        if (((speicher_entry_order_delta_mandant.Attribute_Name == value1) & (speicher_entry_order_delta_mandant.Delta == bool1))&((speicher_entry_order_delta_mandant.Attribute_Name == value2) & (speicher_entry_order_delta_mandant.Delta == bool2))).any(): 
            if ((speicher_entry_order_delta_mandant.Attribute_Name == value3) & (speicher_entry_order_delta_mandant.Delta == False)).any():
                    counter_vom += 1

    #if((counter_vom == 0) | (denominator_order_delta_mandant_num == 0 & counter_vom == 0)):
    try:
        return counter_vom/denominator_order_delta_mandant_num
        #return ([counter_vom/denominator_order_delta_mandant_num,counter_vom,denominator_order_delta_mandant_num])
    except:
         return pd.NA

In [9]:
def get_data_values_two_booleans2(df, items):    
    counter_vom=0

    df_order_mandant_delta = df.loc[(df['Attribute_Name'] == items[0])&(df['Delta'] == items[1])]
    df_order_delta_new = df_order_mandant_delta.loc[(df_order_mandant_delta['Attribute_Name'] == items[2])&(df['Delta'] == items[3])]
    df_order_delta_mandant_documentid = df_order_delta_new[['DocumentID']]
    order_delta_mandant_list = df_order_delta_mandant_documentid.values.tolist()
    order_delta_mandant_list_merged = list(itertools.chain.from_iterable(order_delta_mandant_list))
    denominator_order_delta_mandant_num = len(order_delta_mandant_list_merged)


    df_invoicenumber_order_delta_mandant = df.loc[(df['Attribute_Name'] == items[0])|(df['Attribute_Name'] == items[2])|(df['Attribute_Name'] == items[4])]
    for entry_order_delta_mandant in order_delta_mandant_list_merged:
        speicher_entry_order_delta_mandant = df_invoicenumber_order_delta_mandant[df_invoicenumber_order_delta_mandant.DocumentID == ''.join(str(entry_order_delta_mandant))]
        if (((speicher_entry_order_delta_mandant.Attribute_Name == items[0]) & (speicher_entry_order_delta_mandant.Delta == items[1]))&((speicher_entry_order_delta_mandant.Attribute_Name == items[2]) & (speicher_entry_order_delta_mandant.Delta == items[3]))).any(): 
            if ((speicher_entry_order_delta_mandant.Attribute_Name == items[4]) & (speicher_entry_order_delta_mandant.Delta == False)).any():
                    counter_vom += 1

    try:
        return counter_vom/denominator_order_delta_mandant_num, denominator_order_delta_mandant_num
    except:
         return pd.NA , pd.NA

In [10]:
def get_single_value(df, value):
    df_ordernum = df.loc[(df['Attribute_Name'] == value)] 
    dist = df_ordernum['Delta'].value_counts(normalize=True)
    try:
        score_false = dist.loc[False]
        if(score_false >= 0):
            score = score_false
        elif(score_false == 0):
            score = dist.loc[True]
    except:
        return pd.NA , pd.NA

        
    return score, len(df_ordernum)



In [11]:
list1 = [['OrderNum',False,'DEBITOR_NUM'],
         ['OrderNum',False,'VENDOR_NUM'],
         ['OrderNum',True,'DEBITOR_NUM'],
         ['VENDOR_NUM',False,'InvoiceNumber'],
         ['VENDOR_NUM',True,'InvoiceNumber'],
         ['VENDOR_NUM',False,'InvoiceDate'],
         ['VENDOR_NUM',True,'InvoiceDate'],
         ['VENDOR_NUM',False,'GrossAmount'],
         ['VENDOR_NUM',True,'GrossAmount'],
         ['VENDOR_NUM',False,'NetAmount1'],
         ['VENDOR_NUM',True,'NetAmount1'],
         ['VENDOR_NUM',False,'VatAmount1'],
         ['VENDOR_NUM',True,'VatAmount1'],
         ]

list2 = [['OrderNum',True,'DEBITOR_NUM',False,'VENDOR_NUM'],
            ['OrderNum',True,'DEBITOR_NUM',True,'VENDOR_NUM'],
            ]

def get_data_values_complete_results(tenant, splitkey,df_new):

    dc_vendors = {}
    dc_frequency = {}

    dc_sorted_df = split_datframe_into_whatever(splitkey, df_new)
    
    df_results = pd.DataFrame(columns=['q(OrderNum)','q(VENDOR_NUM)',
            'q(DEBITOR_NUM|OrderNum)',
            'q(VENDOR_NUM|OrderNum)',
            'q(DEBITOR_NUM|!OrderNum)',
            'q(InvoiceNumber|VENDOR_NUM)',
            'q(InvoiceNumber|!VENDOR_NUM)',
            'q(InvoiceDate|VENDOR_NUM)',
            'q(InvoiceDate|!VENDOR_NUM)',
            'q(GrossAmount|VENDOR_NUM)',
            'q(GrossAmount|!VENDOR_NUM)',
            'q(NetAmount1|VENDOR_NUM)',
            'q(NetAmount1|!VENDOR_NUM)',
            'q(VatAmount1|VENDOR_NUM)',
            'q(VatAmount1|!VENDOR_NUM)',
            'q(VENDOR_NUM | !OrderNum & DEBITOR_NUM)',
            'q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)',
            ])
    
    df_results_frequency= pd.DataFrame(columns=['q(OrderNum)','q(VENDOR_NUM)',
            'q(DEBITOR_NUM|OrderNum)',
            'q(VENDOR_NUM|OrderNum)',
            'q(DEBITOR_NUM|!OrderNum)',
            'q(InvoiceNumber|VENDOR_NUM)',
            'q(InvoiceNumber|!VENDOR_NUM)',
            'q(InvoiceDate|VENDOR_NUM)',
            'q(InvoiceDate|!VENDOR_NUM)',
            'q(GrossAmount|VENDOR_NUM)',
            'q(GrossAmount|!VENDOR_NUM)',
            'q(NetAmount1|VENDOR_NUM)',
            'q(NetAmount1|!VENDOR_NUM)',
            'q(VatAmount1|VENDOR_NUM)',
            'q(VatAmount1|!VENDOR_NUM)',
            'q(VENDOR_NUM | !OrderNum & DEBITOR_NUM)',
            'q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)',
            ])

    for key, df in dc_sorted_df.items():

        df_speicher = pd.DataFrame(columns=['q(OrderNum)','q(VENDOR_NUM)',
            'q(DEBITOR_NUM|OrderNum)',
            'q(VENDOR_NUM|OrderNum)',
            'q(DEBITOR_NUM|!OrderNum)',
            'q(InvoiceNumber|VENDOR_NUM)',
            'q(InvoiceNumber|!VENDOR_NUM)',
            'q(InvoiceDate|VENDOR_NUM)',
            'q(InvoiceDate|!VENDOR_NUM)',
            'q(GrossAmount|VENDOR_NUM)',
            'q(GrossAmount|!VENDOR_NUM)',
            'q(NetAmount1|VENDOR_NUM)',
            'q(NetAmount1|!VENDOR_NUM)',
            'q(VatAmount1|VENDOR_NUM)',
            'q(VatAmount1|!VENDOR_NUM)',
            'q(VENDOR_NUM | !OrderNum & DEBITOR_NUM)',
            'q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)',
            ])
        
        df_speicher2 = pd.DataFrame(columns=['q(OrderNum)','q(VENDOR_NUM)',
            'q(DEBITOR_NUM|OrderNum)',
            'q(VENDOR_NUM|OrderNum)',
            'q(DEBITOR_NUM|!OrderNum)',
            'q(InvoiceNumber|VENDOR_NUM)',
            'q(InvoiceNumber|!VENDOR_NUM)',
            'q(InvoiceDate|VENDOR_NUM)',
            'q(InvoiceDate|!VENDOR_NUM)',
            'q(GrossAmount|VENDOR_NUM)',
            'q(GrossAmount|!VENDOR_NUM)',
            'q(NetAmount1|VENDOR_NUM)',
            'q(NetAmount1|!VENDOR_NUM)',
            'q(VatAmount1|VENDOR_NUM)',
            'q(VatAmount1|!VENDOR_NUM)',
            'q(VENDOR_NUM | !OrderNum & DEBITOR_NUM)',
            'q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)',
            ])
        
        result_list = []
        result_list_frequency = []

        ordernum, ordernum_frequency = get_single_value(df, 'OrderNum')
        result_list.append(ordernum)
        result_list_frequency.append(ordernum_frequency)

        vendornum, vendornum_frequency = get_single_value(df, 'VENDOR_NUM')

        result_list.append(vendornum)
        result_list_frequency.append(vendornum_frequency)

        for items1 in list1:
            value, value_frequency = get_data_values2(df, items1)
            result_list.append(value)
            result_list_frequency.append(value_frequency)

        for items2 in list2:
            twovalues, twovalues_frequency = get_data_values_two_booleans2(df, items2)
            result_list.append(twovalues)
            result_list_frequency.append(twovalues_frequency)
            
        df_speicher.loc[key] = result_list 
        df_speicher2.loc[key] = result_list_frequency
  
        df_results.loc[key] = result_list
        df_results_frequency.loc[key] = result_list_frequency


        
        dc_vendors[key] = df_speicher    
        dc_frequency[key] = df_speicher2

        #display(dc_vendors)
        #display(dc_frequency)
        
    return df_results, df_results_frequency ,dc_sorted_df, dc_vendors, dc_frequency


In [ ]:
# Sollwerte bestimmen

def sollwerte_anhand_wahrscheinlichkeit(df):
    sollwerte = pd.Series()
    for column in df.columns:
        #df.values.tolist()
        probability = []
        count = []
        for entry in df[column]:
            #df.values.tolist()
            if (pd.isnull(entry)):
                continue      
            else:
                probability.append(entry[0])
                count.append(entry[1])

        wholecount = sum(count)
        gesamtwahrscheinlichkeit = sum([p * h / wholecount for p, h in zip(probability, count)])  #p = probability * count/wholcount

        sollwerte[column] = gesamtwahrscheinlichkeit
    return sollwerte

sollwerte = sollwerte_anhand_wahrscheinlichkeit(df_results)
#sollwerte.to_csv('data/sollwerte.csv', index=True, header= True, encoding='utf-8')
print(sollwerte)

In [12]:
sollwerte = pd.read_csv('data/sollwerte.csv', encoding='utf-8')
sollwerte_transposed = sollwerte.set_index('Unnamed: 0').T

display(sollwerte_transposed)

Unnamed: 0,q(OrderNum),q(VENDOR_NUM),q(DEBITOR_NUM|OrderNum),q(VENDOR_NUM|OrderNum),q(DEBITOR_NUM|!OrderNum),q(InvoiceNumber|VENDOR_NUM),q(InvoiceNumber|!VENDOR_NUM),q(InvoiceDate|VENDOR_NUM),q(InvoiceDate|!VENDOR_NUM),q(GrossAmount|VENDOR_NUM),q(GrossAmount|!VENDOR_NUM),q(NetAmount1|VENDOR_NUM),q(NetAmount1|!VENDOR_NUM),q(VatAmount1|VENDOR_NUM),q(VatAmount1|!VENDOR_NUM),q(VENDOR_NUM | !OrderNum & DEBITOR_NUM),q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)
0,1.0,0.9,0.99,0.98,0.95,0.95,0.906,0.9,0.85,0.95,0.9,0.9,0.8,0.9,0.8,0.8,0.5
Gewichtung,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,0.5,2.0,1.0,0.7,0.7,0.7,0.7,1.0,1.0
Comment,Regulären Ausdruck,variable,wenn nicht stammdatenproblem/falsche bestelldaten,wenn nicht stammdatenproblem/falsche bestelldaten,stammdaten (Adressdaten) im beleg/kopfbereich,Autotrainerproblem,generische Rechnungsnummersuche über Schlüssel...,Autotrainerproblem,generische Rechnungsnummersuche über Schlüssel...,Autotrainerproblem,Betragserkennung,Autotrainerproblem,Betragserkennung abhängig von Steuersatz,Autotrainerproblem,Betragserkennung abhängig von Steuersatz,Lieferantenstammdaten und Bestellstammdaten/se...,Lieferantenstammdaten/semantische Klassifizier...


In [13]:
def first_element(row):
    return row[0]

def get_outliers(df_results):
    outliers_dataframe = pd.Series([],dtype=pd.StringDtype())

    for col in sollwerte_transposed:
        #print(sollwerte_transposed[col])
        std = df_results[col].std()
        sollwert = sollwerte_transposed.iloc[0][col]
        gewichtung = sollwerte_transposed.iloc[1][col]
        absoluter_abstand = np.abs(sollwert - df_results[col])
        outliers = df_results[(df_results[col] - sollwert).abs() > 0.1] #2 * std ((1-sollwert)/2)
        #print(outliers.index.tolist())
        if not outliers.empty:
            outliers_dataframe[col] = outliers.index.tolist()
            #print(f"Ausreißer in Spalte {col}:")
            #print(outliers.index)
            #return(col, outliers.index)

    return(outliers_dataframe)

#median = df_results.median()

def sort_outliers(outliers_results, df_results_frequency):
    sorted_outliers_dataframe = pd.Series([],dtype=pd.StringDtype())
    for entry in outliers_results.items():
        filtered_df2 = df_results_frequency[df_results_frequency.index.isin(entry[1])]
        sorted_filtered_df2 = filtered_df2.sort_values(by=entry[0], ascending=False)
        sorted_filtered_df2 = sorted_filtered_df2[entry[0]]
        sorted_outliers_dataframe[entry[0]]  = sorted_filtered_df2.index.tolist()
    #print(df_results_debitor_frequency[entry[0]].sort_values(entry[1]))
    return sorted_outliers_dataframe
    #entry[1]

In [14]:
display(dc_all_results_df_vendor_complete["1100"]["10097"])

NameError: name 'dc_all_results_df_vendor_complete' is not defined

In [15]:
def sort_numbers_by_position(df, sollwerte_transposed ,col_name, threshold):
    count_numbers = {}
    positions = {}
    for i, row in df.iterrows():
        gewichtung = sollwerte_transposed.iloc[1][i]
        for number in row[col_name]:
            if number in count_numbers:
                count_numbers[number] += 1 * gewichtung
            else:
                count_numbers[number] = 1
            if number in positions:
                positions[number].append(row[col_name].index(number))
            else:
                positions[number] = [row[col_name].index(number)]
    
    sorted_counts = sorted(count_numbers.items(), key=lambda x: sum(positions[x[0]]) / len(positions[x[0]]))
    sorted_numbers = [x[0] for x in sorted_counts]
    
    threshold_numbers = [x[0] for x in sorted_counts if x[1] >= threshold]
    
    return sorted_numbers, threshold_numbers

In [127]:
startdate = "'2022-12-20 09:44:23.030'"
enddate = "'2023-06-13 12:28:20.000'"

df_table_cclogattributes = get_table_data_CCLOG('CCLogAttributes', connect_to_db_better(connection_string= 'classconprocessingger.database.windows.net', database = 'T_'+tenant+'')
                                           ,""+startdate+"",""+enddate+"")
df_table_cclogattributes = df_table_cclogattributes.drop(['Zone','Attribute_DataType','LogTimeTicks'], axis=1)
df_table_cclogattributes = df_table_cclogattributes.replace('\n',' ', regex=True)
df_table_cclogattributes = df_table_cclogattributes.replace('\r',' ', regex=True)
df_table_cclogattributes.to_csv('./data/cclogattributes_T_'+tenant+'_reduced.csv', index=False, header= True, encoding='utf-8')


In [78]:
df_scorecard_dataframe = pd.DataFrame(columns=['Mandant','Lieferant','Fehlercode','DocumentID','MissingCode'])
df_scorecard_dataframe.loc[0, 'Mandant'] = '137 Belagswerk Oberland AG'
print(df_scorecard_dataframe)

                      Mandant Lieferant Fehlercode DocumentID MissingCode
0  137 Belagswerk Oberland AG       NaN        NaN        NaN         NaN


In [79]:
score_card = pd.Series([],dtype=pd.StringDtype())
df_scorecard_dataframe = pd.DataFrame(columns=['Mandant','Lieferant','Fehlercode','DocumentID','MissingCode'])

sollwerte = pd.read_csv('data/sollwerte.csv', encoding='utf-8')
sollwerte_transposed = sollwerte.set_index('Unnamed: 0').T
dc_sorted_df_vendor_complete = dict()
dc_all_frequency_df_vendor_complete = {}
dc_all_results_df_vendor_complete = {}

df_results_debitor,df_results_frequency ,dc_sorted_df_debitor, schmutz, schmutz2 = get_data_values_complete_results(tenant, 'DEBITOR_NUM', df_table_cclogattributes)
outliers_results = get_outliers(df_results_debitor)
outliers_results_sorted_debitor = sort_outliers(outliers_results, df_results_frequency)
outliers_results_debitor_frame = outliers_results_sorted_debitor.to_frame()
sorted_counts, high_frequency_numbers_debitor = sort_numbers_by_position(outliers_results_debitor_frame, sollwerte_transposed,0, threshold=3)

bad_vendors = []
count = 0
for entry in high_frequency_numbers_debitor:
    dc_sorted_df_vendor_complete[entry] = {}
    dc_all_results_df_vendor_complete[entry] = {}
    dc_all_frequency_df_vendor_complete[entry] = {}
    df_results_vendor, df_results_frequency_vendor, dc_sorted_df_vendor, dc_vendors, dc_frequency = get_data_values_complete_results(tenant, 'VENDOR_NUM', dc_sorted_df_debitor[entry])
    dc_all_results_df_vendor_complete[entry].update(dc_vendors)
    dc_sorted_df_vendor_complete[entry].update(dc_sorted_df_vendor)
    #display(dc_frequency)
    dc_all_frequency_df_vendor_complete[entry].update(dc_frequency)
    outliers_results_vendor = get_outliers(df_results_vendor)
    outliers_results_sorted_vendor = sort_outliers(outliers_results_vendor, df_results_frequency_vendor)
    outliers_results_vendor_frame = outliers_results_sorted_vendor.to_frame()
    sorted_counts_vendor, high_frequency_numbers_vendor = sort_numbers_by_position(outliers_results_vendor_frame, sollwerte_transposed, 0, threshold=3)
    bad_vendors.append(high_frequency_numbers_vendor)
    print(high_frequency_numbers_vendor)

    for entry_high_frequency_numbers_vendor in high_frequency_numbers_vendor:
        df_scorecard_dataframe.loc[count, 'Mandant'] = entry
        df_scorecard_dataframe.loc[count, 'Lieferant'] = entry_high_frequency_numbers_vendor 
        count += 1
        
    score_card[entry]  = high_frequency_numbers_vendor

for entry in list(set(df_results_debitor.index) - set(high_frequency_numbers_debitor)):
    dc_sorted_df_vendor_complete[entry] = {}
    dc_all_results_df_vendor_complete[entry] = {}
    df_results_vendor, df_results_frequency_vendor, dc_sorted_df_vendor, dc_vendors,dc_frequency = get_data_values_complete_results(tenant, 'VENDOR_NUM', dc_sorted_df_debitor[entry])
    dc_all_results_df_vendor_complete[entry].update(dc_vendors)
    dc_sorted_df_vendor_complete[entry].update(dc_sorted_df_vendor)
    dc_all_frequency_df_vendor_complete[entry].update(dc_frequency)


['16042', '16021', '10117', '18073', '17505', '17424', '16026', '18360', '17488', '17368', '16916', '18290', '18465', '102204', '18176', '17476', '18414', '17510', '17508', '17380', '17492', '17948', '17415', '18116', '18451', '18135', '18224', '18329', '18340', '18349', '18441', '16615', '18380', '18274', '18322', '17942', '18402', '16781', '18418', '18326', '18177', '16741', '18435', '18129', '18419', '18434', '18049', '17662', '18004', '18265', '17980', '18024', '17209', '17223', '18461', '16635', '18320', '17074', '102330', '18388', '18346', '17367', '17420', '17300', '16664', '17947', '18083', '17356', '18080', '18389', '15664', '18226', '18077', '14474', '17531', '17094', '18310', '17237', '17461', '18460', '101703', '18167', '17421', '18153', '17342', '18263', '18319', '17477']
['16021', '15300', '10908', '15569', '15054', '17514', '17206', '10100', '17368', '17304', '10252', '15675', '17930', '16169', '14138', '16545', '11456', '17489', '10008', '17459', '18159', '15469', '1599

In [19]:
display(dc_all_frequency_df_vendor_complete["1200"]["15779"])


,q(OrderNum),q(VENDOR_NUM),q(DEBITOR_NUM|OrderNum),q(VENDOR_NUM|OrderNum),q(DEBITOR_NUM|!OrderNum),q(InvoiceNumber|VENDOR_NUM),q(InvoiceNumber|!VENDOR_NUM),q(InvoiceDate|VENDOR_NUM),q(InvoiceDate|!VENDOR_NUM),q(GrossAmount|VENDOR_NUM),q(GrossAmount|!VENDOR_NUM),q(NetAmount1|VENDOR_NUM),q(NetAmount1|!VENDOR_NUM),q(VatAmount1|VENDOR_NUM),q(VatAmount1|!VENDOR_NUM),q(VENDOR_NUM | !OrderNum & DEBITOR_NUM),q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)
15779,<NA>,9,<NA>,<NA>,<NA>,1,8,1,8,1,8,1,8,1,8,<NA>,<NA>


In [24]:
sollwerte = pd.read_csv('data/sollwerte.csv', encoding='utf-8')
sollwerte_transposed = sollwerte.set_index('Unnamed: 0').T
sollwerte_json = sollwerte_transposed.iloc[0].to_json()
print(sollwerte_json)
import csv 

def generate_json_object():
    json_object = {}

    with open('data/sollwerte.csv', 'r', encoding='utf-8') as file:
        csv_data = csv.DictReader(file)
        
        for row in csv_data:
            key = row['']
            value = row['Comment']
            
            if key and value:
                json_object[key] = value
    
    return json_object

json_data = generate_json_object()
json_string = json.dumps(json_data, indent=4)
print(json_string) 

{"q(OrderNum)":1.0,"q(VENDOR_NUM)":0.9,"q(DEBITOR_NUM|OrderNum)":0.99,"q(VENDOR_NUM|OrderNum)":0.98,"q(DEBITOR_NUM|!OrderNum)":0.95,"q(InvoiceNumber|VENDOR_NUM)":0.95,"q(InvoiceNumber|!VENDOR_NUM)":0.906,"q(InvoiceDate|VENDOR_NUM)":0.9,"q(InvoiceDate|!VENDOR_NUM)":0.85,"q(GrossAmount|VENDOR_NUM)":0.95,"q(GrossAmount|!VENDOR_NUM)":0.9,"q(NetAmount1|VENDOR_NUM)":0.9,"q(NetAmount1|!VENDOR_NUM)":0.8,"q(VatAmount1|VENDOR_NUM)":0.9,"q(VatAmount1|!VENDOR_NUM)":0.8,"q(VENDOR_NUM | !OrderNum & DEBITOR_NUM)":0.8,"q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)":0.5}
{
    "q(OrderNum)": "Regul\u00e4ren Ausdruck",
    "q(VENDOR_NUM)": "variable",
    "q(DEBITOR_NUM|OrderNum)": "wenn nicht stammdatenproblem/falsche bestelldaten",
    "q(VENDOR_NUM|OrderNum)": "wenn nicht stammdatenproblem/falsche bestelldaten",
    "q(DEBITOR_NUM|!OrderNum)": "stammdaten (Adressdaten) im beleg/kopfbereich",
    "q(InvoiceNumber|VENDOR_NUM)": "Autotrainerproblem",
    "q(InvoiceNumber|!VENDOR_NUM)": "generische Rechnungsn

In [56]:
df_table_cclogvariants = get_table_data_ALL('CCLOGVARIANTS', connect_to_db_better(connection_string= 'classconprocessingger.database.windows.net', database = 'T_'+tenant+''))
df_table_cclogautotrain = get_table_data_CCLOGAUTOTRAIN('CCLOGAUTOTRAIN', connect_to_db_better(connection_string= 'classconprocessingger.database.windows.net', database = 'T_'+tenant+''),""+startdate+"",""+enddate+"")

df_table_cclogvariants = df_table_cclogvariants.replace('\n',' ', regex=True)
df_table_cclogautotrain = df_table_cclogautotrain.replace('\r',' ', regex=True)
df_table_cclogvariants = df_table_cclogvariants.replace('\n',' ', regex=True)
df_table_cclogautotrain = df_table_cclogautotrain.replace('\r',' ', regex=True)

df_table_cclogvariants = df_table_cclogvariants.drop(['ID', 'ATTRIB_STRUCTURE','DCMODULE_NAME'], axis=1)
df_table_cclogvariants = df_table_cclogvariants.rename({'TRAINING_ID': 'ID'}, axis=1)


In [125]:
merged_df = pd.merge(df_table_cclogautotrain, df_table_cclogvariants, on='ID')
merged_df[['VENDOR', 'VENDOR_NUM', 'CHANGED_TYPE']] = merged_df['ATTRIB_PATH'].str.split('.', expand=True)


data_dict = merged_df.set_index('VENDOR_NUM').T.to_dict()
json_obj = {key: value for key, value in data_dict.items()}

display(merged_df)
#save_file = open("data/savedata.json", "w")  
#json.dump(json_obj, save_file, indent = 6)  
#save_file.close()  

# print(json_obj)
# # JSON-Objekt als String ausgeben
# json_str = json.dumps(json_obj)

# Ergebnis anzeigen
#print(json_str)

C:\Users\lete\AppData\Local\Temp\ipykernel_18116\1243603414.py:5: UserWarning:

DataFrame columns are not unique, some columns will be omitted.



,ID,START_TIME,STOP_TIME,START_TIME_TICKS,CONF_FILE,CATEGORIES_ANALYZED,TEMPLATES_INSERTED,TEMPLATES_DELETED,CLIENT_FIELD,SCORE,TESTED_COUNT,ATTRIB_PATH,STATUS,DELETING_TASK_ID,VENDOR,VENDOR_NUM,CHANGED_TYPE
0,3A52B15A-C122-4921-882A-193204F84D81,16.03.2023 14:02:12,16.03.2023 14:02:12,638145721329724270,,2.0,1.0,0.0,17515,0.428571,12.0,vendor.17515.TrustedAmount,deleted,4E86954F-04E4-4AC8-9AFB-4A6E34DDCFBA,vendor,17515,TrustedAmount
1,315CF0BC-D815-41A5-BF04-8F058FE78CD6,21.03.2023 09:39:11,21.03.2023 09:39:11,638149883519738651,,3.0,2.0,1.0,17942,0.142857,7.0,vendor.17942.invoicenumber,inserted,None,vendor,17942,invoicenumber
2,315CF0BC-D815-41A5-BF04-8F058FE78CD6,21.03.2023 09:39:11,21.03.2023 09:39:11,638149883519738651,,3.0,2.0,1.0,17942,0.428571,7.0,vendor.17942.TrustedAmount,inserted,None,vendor,17942,TrustedAmount
3,8DDA731C-4D0B-42BB-963F-12D34ECC2032,21.03.2023 13:01:22,21.03.2023 13:01:22,638150004820829102,,1.0,1.0,0.0,16204,0.153846,13.0,vendor.16204.TrustedAmount,deleted,3084CED9-5ECB-4E4F-8590-A120D2643992,vendor,16204,TrustedAmount
4,D00B88EA-7C2E-4F20-8F32-F212DF4F2F0B,22.03.2023 09:39:05,22.03.2023 09:39:05,638150747451459823,,3.0,1.0,0.0,17485,0.733333,15.0,vendor.17485.TrustedAmount,inserted,None,vendor,17485,TrustedAmount
5,363B54B6-4282-468C-8C4F-684B6EBF3D9A,28.03.2023 11:33:25,28.03.2023 11:33:25,638156000051181974,,7.0,1.0,0.0,10060,0.400000,4.0,vendor.10060.TrustedAmount,inserted,None,vendor,10060,TrustedAmount
6,B51411F6-9BB7-4D14-AB5A-2DC8FF95C21F,30.03.2023 14:14:14,30.03.2023 14:14:14,638157824546582865,,3.0,1.0,1.0,17380,0.833333,12.0,vendor.17380.TrustedAmount,inserted,None,vendor,17380,TrustedAmount
7,BFB9F951-D570-4A59-A87D-5CCB00B3161F,13.04.2023 09:44:41,13.04.2023 09:44:41,638169758817916463,,4.0,1.0,0.0,18030,1.000000,3.0,vendor.18030.invoicenumber,inserted,None,vendor,18030,invoicenumber
8,803326B3-1638-468E-A68B-DE2C2BB03197,27.03.2023 11:18:09,27.03.2023 11:18:09,638155126891551177,,1.0,1.0,0.0,15138,0.866667,15.0,vendor.15138.TrustedAmount,inserted,None,vendor,15138,TrustedAmount
9,B1BF7590-000F-44CA-AF6B-B0AA42C5054E,03.04.2023 09:13:50,03.04.2023 09:13:50,638161100309560802,,2.0,1.0,0.0,18274,1.000000,2.0,vendor.18274.invoicenumber,inserted,None,vendor,18274,invoicenumber


In [123]:
#display(dc_sorted_df_vendor_complete["1100"]["10041"])
from collections import defaultdict

flat_dict = defaultdict(pd.DataFrame)


for mandant, lieferanten_dict in dc_sorted_df_vendor_complete.items():
    for lieferant, df in lieferanten_dict.items():
        if not flat_dict[lieferant].empty:
            flat_dict[lieferant] = pd.concat([flat_dict[lieferant], df])
        else:
            flat_dict[lieferant] = df
        
flat_dict = dict(flat_dict)

for df_name, df in flat_dict.items():
    sorted_df = df.sort_values(by='LogTime', ascending=False)

    unique_ids = []

    for index, row in sorted_df.iterrows():
        if row['DocumentID'] not in unique_ids:
            unique_ids.append(row['DocumentID'])
        if len(unique_ids) == 5:
            break


    print(df_name, unique_ids)


16021 ['523d9df3-17c3-4397-b0f1-d13c774e6e26', '9926f88c-f3b5-491d-aa44-531feb5f8141', '194313f4-c23c-499a-ada4-df450a82b6d1', '0e502898-7030-477d-9faf-f4e202032c96', '036f66d8-da11-4606-afb3-9639463a0dd5']
17223 ['b10494b5-b3e9-47ef-8d80-75ec42254ef7', '98c0576c-5f7b-47dd-ae3d-2521878384d8', 'e09e3b01-e354-46ca-98fe-6e8dfb35716e', 'c0b9b91b-5f07-4cfe-a424-128071981a83', '2b4e753b-0a56-46f6-a270-8019406f797c']
10117 ['de5a6df1-32fa-46cb-8b3a-860d6716e55e', '0021ab8e-5d77-4135-b787-5a4f41cef1e0', '25a02010-b234-4fc4-8672-5a5aa289e411', '46f11f08-294f-4c3f-a72f-272ecd56be01', '046c42c5-d8c3-44e3-9dbb-83e33104e933']
17367 ['0331710d-9b5b-4ce0-859a-d6e85f4ace23', '61bd4d40-2d68-4c5c-92fb-06a170916151', '96580fca-91a1-43e8-ac05-7654217bc78a', '3c155c9a-3748-4eec-9714-45622744ad4e', 'ef280ebc-ccc0-49e1-810f-d898adcf16bd']
17947 ['0d8f806e-7ff3-43be-af5d-e76101105d05']
16049 ['384d1b64-c97a-4cef-bfba-70a26b07de59', '71c62c51-cb24-49a6-aae9-d6e49463c67e', 'd85cf47b-9226-43d3-a462-130ec2a3de9c'

In [25]:
df_table_ccvendors = get_table_data_ALL('CC_VENDORS', connect_to_db_better(connection_string= 'classconprocessingger.database.windows.net', database = 'T_'+tenant+''))
df_table_ccvendors_bank = get_table_data_ALL('CC_VENDOR_BANK', connect_to_db_better(connection_string= 'classconprocessingger.database.windows.net', database = 'T_'+tenant+''))

df_table_ccvendors = df_table_ccvendors.replace('\n',' ', regex=True)
df_table_ccvendors = df_table_ccvendors.replace('\r',' ', regex=True)
df_table_ccvendors_bank = df_table_ccvendors_bank.replace('\n',' ', regex=True)
df_table_ccvendors_bank = df_table_ccvendors_bank.replace('\r',' ', regex=True)

In [26]:
def check_for_eap_error(vendor_num, debitor):  
        df_sort = dc_sorted_df_vendor_complete[debitor][vendor_num]
        df_sort_vendor = df_sort[((df_sort['Attribute_Name'] == 'VENDOR_NUM') & (df_sort['Delta'] == False))]
        documentid_list = df_sort_vendor[['DocumentID']].values.tolist()
        merged = list(itertools.chain.from_iterable(documentid_list))
        df_sorted = df_sort[df_sort['DocumentID'].isin(merged)]

        df_sort_new = df_sorted[((df_sorted['Attribute_Name'] == 'VatAmount1') & (df_sorted['Delta'] == True)) | (df_sorted['Attribute_Name'] == 'NetAmount1') & (df_sorted['Delta'] == True) | (df_sorted['Attribute_Name'] == 'VatRate1') & (df_sorted['Delta'] == True)]

        unique_documentid = df_sort_new['DocumentID'].unique()
        without_vat = [x for x in merged if x not in unique_documentid]
        
        for document_id in without_vat:
                df_sorted = df_sort[(df_sort["DocumentID"] == document_id) & (df_sort["Delta"] == True)]     
                #display(df_sorted)     

        wrong_documentids = []
        error_codes = []

        for entry in unique_documentid:
                sorted_by_documentid = df_sort_new.loc[(df_sort_new["DocumentID"] == entry)]
                len_sorted_by_documentid = len(sorted_by_documentid)
                sorted_by_documentid = sorted_by_documentid.loc[(sorted_by_documentid["Attribute_Name"] == 'VatAmount1')]
                if not sorted_by_documentid.empty:
                        if 0.0 < abs(float(sorted_by_documentid["Attribute_After"]) - float(sorted_by_documentid["Attribute_Before"])) <= 0.05:
                                #print('Bei: '+entry+' Rundungsfehler')
                                x=1

                if not sorted_by_documentid.empty:
                        wrong_documentids.append(entry)
                        error_codes.append(len_sorted_by_documentid)

        return wrong_documentids, error_codes


In [ ]:
# VERALTET NICHT MEHR NOTWENDIG
count = 0
score_card_missing_vendor_vat_registration_id = pd.Series([],dtype=pd.StringDtype())

#score_card_missing_vendor_vat_registration_id = pd.DataFrame()
fehlercount = 0
bad_documents_counter = pd.Series([],dtype=pd.StringDtype()) 
for entry in score_card:
    wrong_number = []
    wrong_documentids = []

    for item in entry:
        #wrong_documentids.append(check_for_eap_error(item, score_card.index[count]))
        wrong_documentids = (check_for_eap_error(item, score_card.index[count]))
        entry_wrong_number_vat_registration_id = df_table_ccvendors[(df_table_ccvendors['COMPANY_NUM'] == score_card.index[count]) & (df_table_ccvendors["VENDOR_NUM"] == item)& ~(df_table_ccvendors["VENDOR_VAT_REGISTRATION_ID"] == '')]
        entry_wrong_number_registration_id = df_table_ccvendors[(df_table_ccvendors['COMPANY_NUM'] == score_card.index[count]) & (df_table_ccvendors["VENDOR_NUM"] == item)& ~(df_table_ccvendors["VENDOR_REGISTRATION_ID"] == '')] 
        entry_wrong_number_iban = df_table_ccvendors_bank[(df_table_ccvendors_bank['COMPANY_NUM'] == score_card.index[count]) & (df_table_ccvendors_bank["VENDOR_NUM"] == item)& ~(df_table_ccvendors_bank["IBAN"] == '')]
        if entry_wrong_number_vat_registration_id.empty: 
            if item not in wrong_number:
                wrong_number.append(item) 
            fehlercount += 100
        if entry_wrong_number_registration_id.empty: 
            if item not in wrong_number:
                wrong_number.append(item)
            fehlercount += 10
        if entry_wrong_number_iban.empty: 
            if item not in wrong_number:
                wrong_number.append(item)
            fehlercount += 1

        wrong_number.append(fehlercount)
        if wrong_documentids:
            bad_documents_counter[score_card.index[count]] = len(wrong_documentids)
            wrong_number.append(wrong_documentids)

        fehlercount = 0
    #print(wrong_documentids)

    score_card_missing_vendor_vat_registration_id[score_card.index[count]] = wrong_number
    count += 1

print(score_card_missing_vendor_vat_registration_id)
print(score_card_missing_vendor_vat_registration_id.to_json('data/scorecard_verbesserung.json', index=True))


In [27]:
current_mandant = None
current_lieferant = None

# Schleife über die Spalten des DataFrames
for column in df_scorecard_dataframe.columns:
    if column == 'Mandant':
        current_mandant = df_scorecard_dataframe[column]
    elif column == 'Lieferant':
        current_lieferant = df_scorecard_dataframe[column]

i = 0

while i < len(current_mandant):
    fehlercount = 0
    wrong_documentids, error_codes = (check_for_eap_error(current_lieferant[i], current_mandant[i]))
    entry_wrong_number_vat_registration_id = df_table_ccvendors[(df_table_ccvendors['COMPANY_NUM'] == current_mandant[i]) & (df_table_ccvendors["VENDOR_NUM"] == current_lieferant[i])& ~(df_table_ccvendors["VENDOR_VAT_REGISTRATION_ID"] == '')]
    entry_wrong_number_registration_id = df_table_ccvendors[(df_table_ccvendors['COMPANY_NUM'] == current_mandant[i]) & (df_table_ccvendors["VENDOR_NUM"] == current_lieferant[i])& ~(df_table_ccvendors["VENDOR_REGISTRATION_ID"] == '')] 
    entry_wrong_number_iban = df_table_ccvendors_bank[(df_table_ccvendors_bank['COMPANY_NUM'] == current_mandant[i]) & (df_table_ccvendors_bank["VENDOR_NUM"] == current_lieferant[i])& ~(df_table_ccvendors_bank["IBAN"] == '')]
    

    if entry_wrong_number_vat_registration_id.empty: 
        fehlercount += 100
    if entry_wrong_number_registration_id.empty: 
        fehlercount += 10
    if entry_wrong_number_iban.empty: 
        fehlercount += 1
    
    df_scorecard_dataframe.loc[i, 'Fehlercode'] = fehlercount
    df_scorecard_dataframe.loc[i, 'DocumentID'] = wrong_documentids
    df_scorecard_dataframe.loc[i, 'MissingCode'] = error_codes

    i+=1

display(df_scorecard_dataframe)
df_scorecard_dataframe.to_json('data/scorecard_df.json', orient="records")

,Mandant,Lieferant,Fehlercode,DocumentID,MissingCode
0,2900,16042,10,[866053da-3e33-42f1-85d1-380161983c34],[3]
1,2900,16021,111,"[254fbf60-5603-4c57-9242-08dea0b82852, ba73ec4...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
2,2900,10117,10,[],[]
3,2900,18073,111,[],[]
4,2900,17505,110,[eb573182-7c61-42de-9569-69d21cf55c3e],[3]
5,2900,17424,110,[],[]
6,2900,16026,10,[],[]
7,2900,18360,111,[],[]
8,2900,17488,110,[],[]
9,2900,17368,10,[],[]


In [111]:
df_results_debitor["Mandant"] = df_results_debitor.index
df_results_debitor.to_json('data/results_debitor.json', orient="records")

In [82]:
df = pd.read_csv('data/cclogattributes_T'+tenant+'_reduced.csv', encoding='utf-8')
df_sorted = df.sort_values(by='DocumentID')
# Finde die Indizes, an denen sich der Eintrag in "Dokument" ändert
document_changes = df_sorted['DocumentID'].ne(df_sorted['DocumentID'].shift())
previous_document = None
bad_documents = 0
whole_documents = 0
for index, row in df_sorted.iterrows():
    count = 0
    if document_changes.iloc[index]:
        whole_documents += 1
        previous_document = row['DocumentID']
    if row['Attribute_Name'] == 'VatAmount1' and row['Delta'] == True:
        count += 1
    if row['Attribute_Name'] == 'NetAmount1' and row['Delta'] == True:
        count += 1
    if row['Attribute_Name'] == 'VatRate1' and row['Delta'] == True:
        count += 1
    if row['Attribute_Name'] == 'InvoiceNumber' and row['Delta'] == True:
        count += 1
    if row['Attribute_Name'] == 'InvoiceDate' and row['Delta'] == True:
        count += 1
    if(count>=1):
        bad_documents += 1

print(bad_documents,whole_documents)


1413 1932


In [48]:
def get_pos_data(df):
    documentid_list = df[['DocumentID']].values.tolist()
    unique_data = [list(x) for x in set(tuple(x) for x in documentid_list)]
    res = list(map(''.join, unique_data))
    len_unique_docids = len(unique_data)
    for entry in res:
        condition = df['DocumentID'] == entry
        result = df.loc[condition]




In [11]:
import pandas as pd
df = pd.read_csv('data/cclogattributes_T000ht3_reduced.csv', encoding='utf-8')

df_sorted_without_vendor_bad = df[~((df['Attribute_Name'] == 'VENDOR_NUM') & (df['Delta'] == True))]
df_sorted_without_vendor_bad = df_sorted_without_vendor_bad[df_sorted_without_vendor_bad['DocumentID'].isin(df_sorted_without_vendor_bad[df_sorted_without_vendor_bad['Attribute_Name'] == 'VENDOR_NUM']['DocumentID'])]
attribute_names = ['InvoiceDate','InvoiceNumber','GrossAmount']
attribute_names_new = ['VatRate1','NetAmount1','VatAmount1']
filtered_df = df_sorted_without_vendor_bad[(df_sorted_without_vendor_bad['Attribute_Name'].isin(attribute_names_new)) & (df_sorted_without_vendor_bad['Delta'] == True)]
grouped_attributes = filtered_df.groupby('DocumentID')['Attribute_Name'].agg(lambda x: ', '.join(sorted(set(x))))
combination_counts = grouped_attributes.value_counts()
x = filtered_df['Attribute_Name'].value_counts()
print(x)
print(combination_counts)

grouped_df = filtered_df.groupby(df_sorted_without_vendor_bad['DocumentID'].ne(df_sorted_without_vendor_bad['DocumentID'].shift()).cumsum())
count_per_group = grouped_df.size()
bad_documents = len(count_per_group)
whole_documents = df_sorted_without_vendor_bad['DocumentID'].nunique()
value = (x[0]/whole_documents)
good_documents = whole_documents-bad_documents

print('Häufigkeit: '+str(value)+' %')
if(value <= 0.01):
    print('Smartinvoice Fehler')
#print(good_documents, whole_documents, x[0])

VatAmount1    1231
NetAmount1     692
VatRate1       365
Name: Attribute_Name, dtype: int64
VatAmount1                          657
NetAmount1, VatAmount1, VatRate1    359
NetAmount1, VatAmount1              212
NetAmount1                          119
VatAmount1, VatRate1                  3
NetAmount1, VatRate1                  2
VatRate1                              1
Name: Attribute_Name, dtype: int64
Häufigkeit: 0.2041797976447172 %


# Correlation Analysis


In [53]:
def handle_non_numerical_data_correlation(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df


bc = handle_non_numerical_data_correlation(df_table_cclogattributes)
#display(bc)
display(bc.corrwith(bc.iloc[0], axis=1))

corr = bc.corr(method = 'pearson')
corr
#plt.figure(figsize=(10,8), dpi =500)
#sns.heatmap(corr,annot=True,fmt=".2f", linewidth=.5)
#plt.show()

0         1.000000
1         0.999880
2         0.999895
3         0.999853
4         0.999756
            ...   
174147    0.999872
174148    0.999898
174149    0.999743
174150    0.999912
174151    0.999801
Length: 174152, dtype: float64

,DocumentID,Attribute_Name,Attribute_Before,Attribute_After,Delta,Type
DocumentID,1.000000,0.000423,-0.006891,-0.007502,-0.009454,-0.009810
Attribute_Name,0.000423,1.000000,0.011665,0.012650,0.013810,0.002291
Attribute_Before,-0.006891,0.011665,1.000000,1.000000,-0.002284,-0.002284
Attribute_After,-0.007502,0.012650,1.000000,1.000000,0.005262,0.002802
Delta,-0.009454,0.013810,-0.002284,0.005262,1.000000,0.948129
Type,-0.009810,0.002291,-0.002284,0.002802,0.948129,1.000000


In [ ]:
# Gruppieren und zählen der eindeutigen Kombinationen
# Berechnung der Korrelationswahrscheinlichkeit
from scipy.stats import pearsonr

# Korrelationsanalyse

# Datensatz einlesen
df = df_table_cclogattributes

df["Attribute_Before"] = pd.to_numeric(df["Attribute_Before"], errors="coerce")
df["Attribute_After"] = pd.to_numeric(df["Attribute_After"], errors="coerce")

# Entferne Zeilen mit NaN-Werten in den relevanten Spalten
df = df.dropna(subset=["Attribute_Before", "Attribute_After"])

# Korrelationsanalyse durchführen
correlation_data = []
for attribute_name in df["Attribute_Name"].unique():
    attribute_data = df[df["Attribute_Name"] == attribute_name]
    if len(attribute_data) > 1:
        correlation_coefficient, _ = pearsonr(attribute_data["Attribute_Before"], attribute_data["Attribute_After"])
        correlation_data.append({
            "Attribute_Name": attribute_name,
            "Correlation_Coefficient": correlation_coefficient
        })

# Ergebnisse ausgeben
correlation_df = pd.DataFrame(correlation_data)
print(correlation_df)

In [ ]:
from scipy.stats import chi2_contingency

def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
    rcorr = r - ((r - 1) ** 2) / (n - 1)
    kcorr = k - ((k - 1) ** 2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

# Korrelationsanalyse mit Cramér's V-Koeffizient
correlation_data = []
for i in range(len(unique_entries)):
    entry_1 = unique_entries[i]
    entry_row_1 = df_table_cclogattributes[df_table_cclogattributes["Attribute_Name"] == entry_1]
    
    for j in range(i+1, len(unique_entries)):
        entry_2 = unique_entries[j]
        entry_row_2 = df_table_cclogattributes[df_table_cclogattributes["Attribute_Name"] == entry_2]
        
        if len(entry_row_1) > 0 and len(entry_row_2) > 0:
            # Kreuztabelle erstellen
            contingency_table = pd.crosstab(entry_row_1["Attribute_Before"], entry_row_2["Attribute_Before"])
            
            # Cramér's V-Koeffizient berechnen
            correlation_coefficient = cramers_v(contingency_table.values)
            
            correlation_data.append({
                "Attribute_Name_1": entry_1,
                "Attribute_Name_2": entry_2,
                "Correlation_Coefficient": correlation_coefficient
            })

# Ergebnisse ausgeben
correlation_df = pd.DataFrame(correlation_data)
print(correlation_df)

In [ ]:
target_attributes = ["Attribute_Name", "Attribute_After"]
target_df = bc[bc["Attribute_Name"].isin(target_attributes)]

# Berechnen Sie die Korrelationen der Zielzeilen mit den anderen Zeilen in "Attribute After"
corr_with_target = target_df["Attribute_After"].corrwith(bc["Attribute_After"])


# Machine Learning


## Data preprocessing

In [54]:
df1 = pd.read_csv('data/cclogattributes_T0012hb.csv', encoding='iso-8859-15')

vergleiche_spalten = lambda x: 'master_data_error' if x['Attribute_Name'] == 'VENDOR_NUM' and x['Attribute_Before'] != x['Attribute_After'] else'good' if x['Delta'] == False and x['Type'] == 0 else 'undefined'
#else 'master_data_error' if x['Attribute_Name'] == 'VENDOR_NAME' and x['Attribute_Before'] != x['Attribute_After'] \
#else 'master_data_error' if x['Attribute_Name'] == 'VENDOR_STR' and x['Attribute_Before'] != x['Attribute_After'] \
#else 'master_data_error' if x['Attribute_Name'] == 'VENDOR_CITY' and x['Attribute_Before'] != x['Attribute_After'] \
#else 'master_data_error' if x['Attribute_Name'] == 'VENDOR_ZIP_CODE' and x['Attribute_Before'] != x['Attribute_After'] \

df1['Suggestion'] = df1.apply(vergleiche_spalten, axis=1)

df1 = df1.drop('LogTimeTicks', axis=1)
#df = df.drop('DocumentID', axis=1)

df1.to_csv('data/after_preprocessing.csv', index=False, header= True, encoding='iso-8859-15')

X = df1.loc[ : , df1.columns != 'Suggestion']
Y = df1.loc[ : , df1.columns == 'Suggestion']


In [20]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

#X = handle_non_numerical_data(X)


def Encoder(df):
          columnsToEncode = list(df.select_dtypes(include=['category','object','bool']))
          le = LabelEncoder()
          for feature in columnsToEncode:
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df


#Y = Encoder(Y)
#print(Y)
#pd.DataFrame(Y).to_csv("data/after_numerical.csv")


#print(X)
#X.shape[0]
#Y.shape[0]

In [4]:
#Create training and testing samples
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=None, shuffle=False)

pd.DataFrame(y_train).to_csv("data/y_train.csv")

## Create Model and Train

In [5]:
def create_model(optimizer='adam', init='uniform'):
    # create model
    if verbose: print("**Create model with optimizer: %s; init: %s" % (optimizer, init) )
    model = Sequential()
    model.add(Dense(16, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu')) #
    model.add(Dense(512, kernel_initializer=init, activation='relu'))
    model.add(Dense(256, kernel_initializer=init, activation='relu'))
    model.add(Dense(3, kernel_initializer=init, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['acc']) #keras.optimizers.Adadelta() and sparse_categorical_accuracy
    return model

In [46]:
run_gridsearch = False

if run_gridsearch:
    
    start_time = time.time()
    if verbose: print (time.strftime( "%H:%M:%S " + "GridSearch started ... " ) )
    optimizers = ['rmsprop'] # , 'adam'
    inits = ['glorot_uniform'] #, 'normal', 'uniform'
    epochs = [5, 50, 100, 300]
    batches = [5, 32, 64]
    
    model = KerasClassifier(build_fn=create_model, verbose=verbose)
    
    param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
    grid = GridSearchCV(estimator=model, param_grid=param_grid)
    grid_result = grid.fit(X_train, y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    if verbose: 
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
        elapsed_time = time.time() - start_time  
        print ("Time elapsed: ",timedelta(seconds=elapsed_time))
        
    best_epochs = grid_result.best_params_['epochs']
    best_batch_size = grid_result.best_params_['batch_size']
    best_init = grid_result.best_params_['init']
    best_optimizer = grid_result.best_params_['optimizer']
    
else:
    # best paramters
    best_epochs = 300
    best_batch_size = 64
    best_init = 'glorot_uniform'
    best_optimizer = 'rmsprop'

In [ ]:
results_df = pd.DataFrame(grid_result.cv_results_)
pd.DataFrame(results_df).to_csv('data/grid_search_results.csv')
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("kernel")
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

In [ ]:
model_scores = results_df.filter(regex=r"split\d*_test_score")

# plot 30 examples of dependency between cv fold and AUC scores
fig, ax = plt.subplots()
sns.lineplot(
    data=model_scores.transpose().iloc[:30],
    dashes=False,
    palette="Set1",
    marker="o",
    alpha=0.5,
    ax=ax,
)
ax.set_xlabel("CV test fold", size=12, labelpad=10)
ax.set_ylabel("Model AUC", size=12)
ax.tick_params(bottom=True, labelbottom=False)
ax.legend(loc=4, prop={'size': 6})

plt.show()


In [ ]:
model_pred = KerasClassifier(build_fn=create_model, optimizer=best_optimizer, init=best_init, epochs=best_epochs, batch_size=best_batch_size, verbose=verbose)
model_pred.fit(X_train, y_train)

prediction = model_pred.predict(X_test)

# Evaluation

In [ ]:
print(classification_report(y_test, prediction, zero_division=False))
accuracy_score(y_test, prediction)

In [ ]:
print(prediction)
pd.DataFrame(prediction).to_csv('data/prediction.csv')

In [50]:
pickle.dump(model_pred, open('models/classification_model.h5', 'wb'))

In [ ]:
with open('models/classification_model.h5', 'rb') as pickle_file:
    content = pickle.load(pickle_file)

prediction = content.predict(X_test)
print(classification_report(y_test, prediction, zero_division=False))


In [ ]:
import pyautogui
import time
while True:
    for i in range(0,100):
        pyautogui.moveTo(10,10)
        pyautogui.leftClick()
        pyautogui.moveTo(1000,1000)
        pyautogui.leftClick()

        time.sleep(2)